In [1]:
import numpy as np
import sys, os
sys.path.append('../../shared/utilsGRF/')
from IPython.display import display
import networkx as nx
import subprocess, glob, re
import matplotlib.pyplot as plt

In [2]:
edges=[]
nnodes_cycle=5
for i in range(1,nnodes_cycle+1):
    if i%nnodes_cycle!=0:
        i2=i+1
        #print(i)
    else:
        i2=i-nnodes_cycle+1
    edges.append([i,"a%d"%i,i2])
    i_=i+nnodes_cycle
    i2_=i2+nnodes_cycle
    edges.append([i_,"alpha%d"%i,i2_])
    edges.append([i,"kb%d-x"%i,i_])
    edges.append([i_,"ku%d"%i,i])
print(edges)

[[1, 'a1', 2], [6, 'alpha1', 7], [1, 'kb1-x', 6], [6, 'ku1', 1], [2, 'a2', 3], [7, 'alpha2', 8], [2, 'kb2-x', 7], [7, 'ku2', 2], [3, 'a3', 4], [8, 'alpha3', 9], [3, 'kb3-x', 8], [8, 'ku3', 3], [4, 'a4', 5], [9, 'alpha4', 10], [4, 'kb4-x', 9], [9, 'ku4', 4], [5, 'a5', 1], [10, 'alpha5', 6], [5, 'kb5-x', 10], [10, 'ku5', 5]]


In [3]:

MTTfolder='../../shared/utilsGRF' #folder with MTT.py 
basename='graph'

parlist=[x[1] for x in edges]
parlist=list(set(parlist))


for pnum,par in enumerate(parlist):
    if '-x' in par:
        parlist[pnum]=parlist[pnum].replace('-x','')
print(parlist)

graphname=os.path.join(MTTfolder,basename+'.txt')
outf=open(graphname,'w')
for x in edges:
    x_str=list(map(str,x))
    outf.write('('+','.join(x_str)+') ')
outf.close()

#execute_MTT(self):

args=["python",os.path.join(MTTfolder,'MTT.py'),graphname]
print("executing MTT")
print(subprocess.check_call(args))


#parse_rhos_from_graphfiles(self):
#this was previously called rhos_from_edges
edges=np.array(edges)
#find files that correspond to the spanning trees
files=glob.glob(os.path.join(MTTfolder,basename+'*'))
pat=re.compile('[0-9]+.txt')
n=0
for f in files:
    if pat.findall(f):
        n+=1
all_rhos=[] 
allGraphs=dict()
allterms=dict()
for i in range(1,n+1):
    fname=os.path.join(MTTfolder,basename+'-%d.txt'%i)

    fi=open(fname,'r')
    rho_expr='rho_%d='%i
    allGraphs[i]=[]
    allterms[i]=[]
    for lnum,l in enumerate(fi.readlines()):
        idxs=np.array(list(map(int,l.strip().split())),dtype=bool)
        subset=edges[idxs]
        expr=[x[1] for x in subset] #labels that multiply together in a term of the rho
        term='*'.join(expr)
        if lnum<1:
            rho_expr=rho_expr+term
        else:
            rho_expr=rho_expr+'+'+term
        allterms[i].append(term)
        #now get Graph
        G=nx.DiGraph()
        for edge in subset:
            n0=int(edge[0])
            n1=int(edge[2])
            par=edge[1]
            G.add_edge(n0,n1,name=par)
            G.add_edge
        allGraphs[i].append(G)
        
            
        

    all_rhos.append(rho_expr)

['kb3', 'ku4', 'a3', 'alpha1', 'ku3', 'alpha5', 'a2', 'kb2', 'ku1', 'kb1', 'a5', 'kb4', 'alpha2', 'ku2', 'alpha3', 'a4', 'a1', 'ku5', 'kb5', 'alpha4']
executing MTT
0


In [4]:
import re
kbpat=re.compile("kb")
alphapat=re.compile("alpha[0-9]")
#alpharepat=re.compile("alpha[0-9]r")
apat=re.compile("^a[0-9]|\*a[0-9]")
kupat=re.compile("ku[0-9]")

#Get mapping between trees rooted at bound state and trees rooted at unbound state:
- if there is a binding edge to the root: change for the unbinding edge and that's it
- if there is not a binding edge to the root: same number of alphas and as but at different places?


In [5]:
#the root+term number is an id for the trees

In [6]:
nnodes=nnodes_cycle
trees_nb=allterms[2*nnodes]
Gs_nb=allGraphs[2*nnodes]
trees_nu=allterms[nnodes]
Gs_nu=allGraphs[nnodes]
dx=5
dy=0.2
#coords={1:(2,0.8),2:(3,0.6),3:(4,0.4),4:(3,0.2),5:(2,0.2),6:(1,0.4),7:(0,0.6),8:(1,0.8),
#        9:(2+dx,0.8+dy),10:(3+dx,0.6+dy),11:(4+dx,0.4+dy),12:(3+dx,0.2+dy),13:(2+dx,0.2+dy),14:(1+dx,0.4+dy),
#        15:(0+dx,0.6+dy),16:(1+dx,0.8+dy)}
coords={1:(2,0.8),2:(3,0.6),3:(4,0.4),4:(2.5,0.2),5:(1.5,0.2),6:(0.5,0.4),7:(1,0.6),8:(1.5,0.7)}
coords2=coords.copy()
for key in coords.keys():
    k0,k1=coords[key]
    coords2[key+nnodes]=(k0+dx,k1+dy)

# Now taking into account that alphas and kus may not be all equal:

In [7]:
import sympy
from sympy.parsing.sympy_parser import parse_expr 

In [15]:
a1,a2,a3,a4,a5,a6,a7,a8=sympy.symbols("a1,a2,a3,a4,a5,a6,a7,a8")
alpha1,alpha2,alpha3,alpha4,alpha5,alpha6,alpha7,alpha8=sympy.symbols("alpha1,alpha2,alpha3,alpha4,alpha5,alpha6,alpha7,alpha8")
ku1,ku2,ku3,ku4,ku5,ku6,ku7,ku8=sympy.symbols("ku1,ku2,ku3,ku4,ku5,ku6,ku7,ku8")
kb1,kb2,kb3,kb4,kb5,kb6,kb7,kb8=sympy.symbols("kb1,kb2,kb3,kb4,kb5,kb6,kb7,kb8")

cs=[a1,a2,a3,a4,a5]
epsilons=[alpha1,alpha2,alpha3,alpha4,alpha5]
kus=[ku1,ku2,ku3,ku4,ku5]
kbs=[kb1,kb2,kb3,kb4,kb5]

In [16]:
import itertools

In [29]:
i=0
j=1
b_i=[]
b_j=[]
for node in range(1,2*nnodes+1):
    for t,term in enumerate(allterms[node]):
            kbs=kbpat.findall(term)
            if len(kbs)==i:
                term_s=parse_expr(term.replace("-x",""))
                b_i.append(term_s)
            elif len(kbs)==j:
                term_s=parse_expr(term.replace("-x",""))
                b_j.append(term_s)
a_i=[]  
a_j=[]
for node in [nnodes,2*nnodes]:
    for t,term in enumerate(allterms[node]):
        kbs=kbpat.findall(term)
        if len(kbs)==i:
            term_s=parse_expr(term.replace("-x",""))
            if node>nnodes:
                a_i.append(epsilons[-1]*term_s)
            else:
                a_i.append(term_s)
        elif len(kbs)==j:
            term_s=parse_expr(term.replace("-x",""))
            if node>nnodes:
                a_j.append(epsilons[-1]*term_s)
            else:
                a_j.append(term_s)   

In [30]:
positive=[]
for term in b_i:
    for term2 in a_j:
        positive.append(term*term2)
negative=[]
for term in b_j:
    for term2 in a_i:
        negative.append(term*term2)
print(len(positive))
print(len(negative))

12400
12400


In [31]:
found=np.zeros(len(positive))
for t,term in enumerate(positive):
    for term2 in negative:
        div=term/term2
        if div==1:
            found[t]=1
        elif div in epsilons:
            found[t]=2

KeyboardInterrupt: 

In [36]:
found=[]
for t,term in enumerate(positive):
    if not t in found:
        for t2 in range(t+1,len(positive)-1):
            if term-positive[t2]==0:
                found.append(t2)
                print(t,t2)
            


0 83
0 169
1 85
1 170
2 86
4 2412
7 89
8 90
11 2172
13 94
13 888
13 970
14 890
15 891
15 1852
15 2176
16 892
17 255
17 576
17 902
17 1131
17 1452
17 1864
17 2178
18 256
18 904
18 1132
19 103
20 101
20 247
20 329
20 2179
20 2263


KeyboardInterrupt: 

In [34]:
found[0:500]

array([1., 1., 1., 1., 0., 1., 1., 1., 1., 1., 1., 0., 0., 1., 1., 0., 0.,
       0., 0., 1., 1., 1., 0., 1., 0., 1., 1., 1., 1., 1., 0., 1., 0., 1.,
       1., 1., 0., 1., 0., 1., 1., 1., 0., 1., 1., 1., 0., 1., 1., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 1., 1., 1., 0.,
       1., 1., 1., 1., 1., 1., 0., 0., 1., 1., 0., 0., 0., 0., 1., 1., 1.,
       0., 1., 0., 1., 1., 1., 1., 1., 0., 1., 0., 1., 1., 1., 0., 1., 0.,
       1., 1., 1., 0., 1., 1., 1., 0., 1., 1., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 1., 1., 1., 1., 0., 1., 1., 1., 1., 1.,
       1., 0., 0., 1., 1., 0., 0., 0., 0., 1., 1., 1., 0., 1., 0., 1., 1.,
       1., 1., 1., 0., 1., 0., 1., 1., 1., 0., 1., 0., 1., 1., 1., 0., 1.,
       1., 1., 0., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0.